# Using `UNION ALL` to address multiple `TABLE`s in the same query

<!--
  ~ Licensed to the Apache Software Foundation (ASF) under one
  ~ or more contributor license agreements.  See the NOTICE file
  ~ distributed with this work for additional information
  ~ regarding copyright ownership.  The ASF licenses this file
  ~ to you under the Apache License, Version 2.0 (the
  ~ "License"); you may not use this file except in compliance
  ~ with the License.  You may obtain a copy of the License at
  ~
  ~   http://www.apache.org/licenses/LICENSE-2.0
  ~
  ~ Unless required by applicable law or agreed to in writing,
  ~ software distributed under the License is distributed on an
  ~ "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY
  ~ KIND, either express or implied.  See the License for the
  ~ specific language governing permissions and limitations
  ~ under the License.
  -->
  
While working with Druid, you may need to bring together two different tables of results together into a single result list, or to treat multiple tables as a single input to a query. This notebook introduces the `UNION ALL` operator, walking through two ways in which this operator can be used to achieve this result: top-level and table-level `UNION ALL`.

## Prerequisites

This tutorial works with Druid 26.0.0 or later.

#### Run using Docker

Launch this tutorial and all prerequisites using the `druid-jupyter` profile of the Docker Compose file for Jupyter-based Druid tutorials. For more information, see [Docker for Jupyter Notebook tutorials](https://druid.apache.org/docs/latest/tutorials/tutorial-jupyter-docker.html).
   
#### Run Druid without Docker

If you do not use the Docker Compose environment, you need the following:

* A running Druid instance, with a `DRUID_HOST` local environment variable containing the servername of your Druid router
* [druidapi](https://github.com/apache/druid/blob/master/examples/quickstart/jupyter-notebooks/druidapi/README.md), a Python client for Apache Druid. Follow the instructions in the Install section of the README file.

### Initialization

Run the next cell to attempt a connection to Druid services. If successful, the output shows the Druid version number.

In [ ]:
import druidapi
import os

if 'DRUID_HOST' not in os.environ.keys():
    druid_host=f"http://localhost:8888"
else:
    druid_host=f"http://{os.environ['DRUID_HOST']}:8888"
    
print(f"Opening a connection to {druid_host}.")
druid = druidapi.jupyter_client(druid_host)

display = druid.display
sql_client = druid.sql
status_client = druid.status

status_client.version

Finally, run the following cell to import the Python JSON module.

In [ ]:
import json

## Using Top-level `UNION ALL` to concatenate result sets

Run the following cell to ingest the wikipedia data example. Once completed, you will see a description of the new table.

You can optionally monitor the ingestion in the Druid console while it runs.

In [ ]:
sql='''
REPLACE INTO "example-wikipedia-unionall" OVERWRITE ALL
WITH "ext" AS (SELECT *
FROM TABLE(
  EXTERN(
    '{"type":"http","uris":["https://druid.apache.org/data/wikipedia.json.gz"]}',
    '{"type":"json"}'
  )
) EXTEND ("isRobot" VARCHAR, "channel" VARCHAR, "timestamp" VARCHAR, "flags" VARCHAR, "isUnpatrolled" VARCHAR, "page" VARCHAR, "diffUrl" VARCHAR, "added" BIGINT, "comment" VARCHAR, "commentLength" BIGINT, "isNew" VARCHAR, "isMinor" VARCHAR, "delta" BIGINT, "isAnonymous" VARCHAR, "user" VARCHAR, "deltaBucket" BIGINT, "deleted" BIGINT, "namespace" VARCHAR, "cityName" VARCHAR, "countryName" VARCHAR, "regionIsoCode" VARCHAR, "metroCode" BIGINT, "countryIsoCode" VARCHAR, "regionName" VARCHAR))
SELECT
  TIME_PARSE("timestamp") AS "__time",
  "isRobot",
  "channel",
  "page",
  "commentLength",
  "countryName",
  "user"
FROM "ext"
PARTITIONED BY DAY
'''

sql_client.run_task(sql)
sql_client.wait_until_ready('example-wikipedia-unionall')
display.table('example-wikipedia-unionall')

You can use `UNION ALL` to append the results of one query with another.

The first query in the cell below, `set1`, returns the ten first edits to any "fr"-like `channel` between midday and 1pm on the 27th June 2016. The second query repeats this but for any "en"-like `channel`.

In [ ]:
sql = '''
WITH
set1 AS (
  SELECT
    __time,
    "channel",
    "page",
    "isRobot"
  FROM "example-wikipedia-unionall"
  WHERE DATE_TRUNC('HOUR', __time) = TIMESTAMP '2016-06-27 12:00:00'
    AND channel LIKE '#fr%'
  ORDER BY __time
  LIMIT 10
  ),
set2 AS (
  SELECT
    __time,
    "channel",
    "page",
    "isRobot"
  FROM "example-wikipedia-unionall"
  WHERE DATE_TRUNC('HOUR', __time) = TIMESTAMP '2016-06-27 12:00:00'
    AND channel LIKE '#en%'
  ORDER BY __time
  LIMIT 10
  )
  
SELECT * from set1
UNION ALL
SELECT * from set2
'''

display.sql(sql)

This is a [top-level](https://druid.apache.org/docs/latest/querying/sql.html#top-level) `UNION` operation. First, Druid calculated `set1` and appended subsequent results sets.

Notice that these results are not in order by time – even though the individual sets did `ORDER BY` time. Druid simply concatenated the two result sets together.

Optionally, run the next cell to show the precise [`EXPLAIN PLAN`](https://druid.apache.org/docs/latest/querying/sql-translation#interpreting-explain-plan-output) for the query. You can see there are two `query` execution plans, one for each subquery. Also, Druid's planning process optimized execution of the outer query.

In [ ]:
print(json.dumps(json.loads(sql_client.explain_sql(sql)['PLAN']), indent=2))

Run next cell to perform another top-level UNION ALL, this time where the sets use `GROUP BY`.

Notice that the aggregates have `AS` to set specific field names.

In [ ]:
sql='''
WITH
set1 AS (
  SELECT
    TIME_FLOOR(__time, 'PT1H') AS "Period",
    countryName,
    AVG(commentLength) AS "Average Comment Size",
    COUNT(DISTINCT "page") AS "Pages"
  FROM "example-wikipedia-unionall"
  WHERE countryName='China'
  GROUP BY 1, 2
  LIMIT 10
  ),
set2 AS (
  SELECT
    TIME_FLOOR(__time, 'PT1H') AS "Episode",
    countryName,
    COUNT(DISTINCT "page") AS "Pages",
    AVG(commentLength) AS "Average Comment Length"
  FROM "example-wikipedia-unionall"
  WHERE countryName='Austria'
  GROUP BY 1, 2
  LIMIT 10
  )

SELECT * from set1
UNION ALL
SELECT * from set2
'''

display.sql(sql)

Look carefully at these results - Druid has simply appended the results from `set2` to `set1` without introducing redundant columns.

* Column name in `set2` (`Period` versus `Episode` and `Average Comment Size` versus `Average Comment Length`) did not result in new columns
* Columns with the same name (`Pages`) did not result in that aggregate being put into same column - Austria's values are simply appended `Average Comment Size`

Run the next cell, which uses explicit column names at the top-level, rather than `*`, to ensure the calculations appear in the right columns in the final result. It also aliases the columns for the results by using `AS`.

In [ ]:
sql='''
WITH
set1 AS (
  SELECT
    TIME_FLOOR(__time, 'PT1H') AS "Period",
    countryName,
    AVG(commentLength) AS "Average Comment Size",
    COUNT(DISTINCT "page") AS "Pages"
  FROM "example-wikipedia-unionall"
  WHERE countryName='China'
  GROUP BY 1, 2
  LIMIT 10
  ),
set2 AS (
  SELECT
    TIME_FLOOR(__time, 'PT1H') AS "Episode",
    countryName,
    COUNT(DISTINCT "page") AS "Pages",
    AVG(commentLength) AS "Average Comment Length"
  FROM "example-wikipedia-unionall"
  WHERE countryName='Austria'
  GROUP BY 1, 2
  LIMIT 10
  )

SELECT "Period", "countryName" AS "Country", "Average Comment Size" AS "Edit Size", "Pages" AS "Unique Pages" from set1
UNION ALL
SELECT "Episode", "countryName", "Average Comment Length", "Pages" from set2
'''

display.sql(sql)

## Using Table-level `UNION ALL` to work with multiple tables

From one source of data, data engineers may create multiple `TABLE` datasources in order to:

* Separate data with different levels of `__time` granularity (ie. the level of summarisation),
* Apply different security to different parts, for example, per tenant,
* Break up the data using filtering at ingestion time, for example, different tables for different HTTP error codes,
* Separate upstream data by the source device or system, for example, different types of IOT device,
* Isolate different periods of time, perhaps with different retention periods.

You can use `UNION ALL` to access _all_ the source data, referencing all the `TABLE` datasources through a sub-query or a `FROM` clause.

The next two cells create two new tables, `example-wikipedia-unionall-en` and `example-wikipedia-unionall-fr`. `example-wikipedia-unionall-en` contains only data for English language channel edits, while `example-wikipedia-unionall-fr` contains only French channels.

Run the next two cells, monitoring the ingestion in the Druid Console as they run.

In [ ]:
sql='''
REPLACE INTO "example-wikipedia-unionall-en" OVERWRITE ALL
WITH "ext" AS (SELECT *
FROM TABLE(
  EXTERN(
    '{"type":"http","uris":["https://druid.apache.org/data/wikipedia.json.gz"]}',
    '{"type":"json"}'
  )
) EXTEND ("isRobot" VARCHAR, "channel" VARCHAR, "timestamp" VARCHAR, "flags" VARCHAR, "isUnpatrolled" VARCHAR, "page" VARCHAR, "diffUrl" VARCHAR, "added" BIGINT, "comment" VARCHAR, "commentLength" BIGINT, "isNew" VARCHAR, "isMinor" VARCHAR, "delta" BIGINT, "isAnonymous" VARCHAR, "user" VARCHAR, "deltaBucket" BIGINT, "deleted" BIGINT, "namespace" VARCHAR, "cityName" VARCHAR, "countryName" VARCHAR, "regionIsoCode" VARCHAR, "metroCode" BIGINT, "countryIsoCode" VARCHAR, "regionName" VARCHAR))
SELECT
  TIME_PARSE("timestamp") AS "__time",
  "isRobot",
  "channel",
  "page",
  "commentLength",
  "countryName",
  "user"
FROM "ext"
WHERE "channel" LIKE '#en%'
PARTITIONED BY DAY
'''

sql_client.run_task(sql)
sql_client.wait_until_ready('example-wikipedia-unionall-en')
display.table('example-wikipedia-unionall-en')

In [ ]:
sql='''
REPLACE INTO "example-wikipedia-unionall-fr" OVERWRITE ALL
WITH "ext" AS (SELECT *
FROM TABLE(
  EXTERN(
    '{"type":"http","uris":["https://druid.apache.org/data/wikipedia.json.gz"]}',
    '{"type":"json"}'
  )
) EXTEND ("isRobot" VARCHAR, "channel" VARCHAR, "timestamp" VARCHAR, "flags" VARCHAR, "isUnpatrolled" VARCHAR, "page" VARCHAR, "diffUrl" VARCHAR, "added" BIGINT, "comment" VARCHAR, "commentLength" BIGINT, "isNew" VARCHAR, "isMinor" VARCHAR, "delta" BIGINT, "isAnonymous" VARCHAR, "user" VARCHAR, "deltaBucket" BIGINT, "deleted" BIGINT, "namespace" VARCHAR, "cityName" VARCHAR, "countryName" VARCHAR, "regionIsoCode" VARCHAR, "metroCode" BIGINT, "countryIsoCode" VARCHAR, "regionName" VARCHAR))
SELECT
  TIME_PARSE("timestamp") AS "__time",
  "isRobot",
  "channel",
  "page",
  "commentLength",
  "countryName",
  "user"
FROM "ext"
WHERE "channel" LIKE '#fr%'
PARTITIONED BY DAY
'''

sql_client.run_task(sql)
sql_client.wait_until_ready('example-wikipedia-unionall-fr')
display.table('example-wikipedia-unionall-fr')

The next cell uses `UNION ALL` in a `WITH` statement that creates `unifiedSource`. This will be a unified source of data for both tables that can then be used in a `SELECT` query.

Druid executes these "[top level](https://druid.apache.org/docs/26.0.0/querying/sql.html#top-level)" `UNION ALL` queries differently to "[table level](https://druid.apache.org/docs/26.0.0/querying/sql.html#table-level)" queries you have used so far. Table level `UNION ALL` makes use of `union` datasources, and it's important that you read the [documentation](https://druid.apache.org/docs/26.0.0/querying/datasource.html#union) to understand the functionality available to you. Operations such as filtering, for example, can only be done in the outer `SELECT` statement on `unifiedSource` in the sample query below.  

Run the following cell to count the number of robot and non-robot edits by channel across both sets.

In [ ]:
sql = '''
WITH unifiedSource AS (
    SELECT
        "__time",
        "isRobot",
        "channel",
        "user",
        "countryName"
    FROM "example-wikipedia-unionall-en"
    UNION ALL
    SELECT
        "__time",
        "isRobot",
        "channel",
        "user",
        "countryName"
    FROM "example-wikipedia-unionall-fr"
    )

SELECT
    "channel",
    COUNT(*) FILTER (WHERE isRobot=true) AS "Robot Edits",
    COUNT (DISTINCT user) FILTER (WHERE isRobot=true) AS "Robot Editors",
    COUNT(*) FILTER (WHERE isRobot=false) AS "Human Edits",
    COUNT (DISTINCT user) FILTER (WHERE isRobot=false) AS "Human Editors"
FROM unifiedSource
GROUP BY 1
'''

display.sql(sql)

## Conclusion

* There are two modes for `UNION ALL` in Druid - top level and table level
* Top level is a simple concatenation, and operations must be done on the source `TABLE`s
* Table level uses a `union` data source, and operations must be done on the outer `SELECT`

## Learn more

* Watch [Plan your Druid table datasources](https://youtu.be/OpYDX4RYLV0?list=PLDZysOZKycN7MZvNxQk_6RbwSJqjSrsNR) by Peter Marshall
* Read about [union](https://druid.apache.org/docs/26.0.0/querying/datasource.html#union) datasources in the documentation
* Read the latest [documentation](https://druid.apache.org/docs/26.0.0/querying/sql.html#union-all) on the `UNION ALL` operator